In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
!nvidia-smi

Thu May  9 10:57:09 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.40.04    Driver Version: 418.40.04    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00005E6A:00:00.0 Off |                    0 |
| N/A   81C    P0   132W / 149W |   6225MiB / 11441MiB |     84%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CUDNN_USE_AUTOTUNE'] = '0'

In [5]:
import tensorflow as tf
config = tf.ConfigProto(intra_op_parallelism_threads=0, 
                        inter_op_parallelism_threads=0,
                        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.15),
                        allow_soft_placement=True)
session = tf.Session(config=config)
tf.keras.backend.set_session(session)

In [6]:
import random
import numpy as np

seed = 11

np.random.seed(seed)
random.seed(seed)
tf.random.set_random_seed(seed)

In [7]:
from fewshot.data_provider.dataset import Dataset
from fewshot.backbones import ConvNet

from fewshot.algorithms.backbone_train import (
    simple_one_layer_cross_entropy_train,
    simple_cosine_layer_cross_entropy_train
)
from fewshot.algorithms.fewshot_test import baseline_fewshot_test
from fewshot.algorithms.fewshot_models import BaselineFewShotModel

import tqdm
from scipy import stats

## Initialize

Data parameters:

Backbone params

Few-shot training parameters:

In [10]:
# fewshot training and testing
n_way = 5
k_shot = 5
fewshot_batch_size = 4
support_epochs = 100
query_size = 16

support_generator_args={
    "size": (int(img_width * 1.15), int(img_height * 1.15)),
    "center": True,  # for center cropping,
    "crop_size": (img_width, img_height),
}

query_generator_args={
    "size": (int(img_width * 1.15), int(img_height * 1.15)),
    "center": True,  # for center cropping,
    "crop_size": (img_width, img_height),
}

n_episodes = 30

In [13]:
checkpoint_dir = '../../../data/few-models/mini_imagenet/checkpoints'
log_dir = '../../../data/few-models/mini_imagenet/logs'

## Data

Create dataset:

In [14]:
dataset_dir = os.path.join(dataset_root, dataset_name)
dataset = Dataset(dataset_dir=dataset_dir, 
                  csv_name=csv_name, 
                  image_size=(img_width, img_height))

Create backbone dataset:

In [15]:
backbone_dataset, val_fewshot_dataset = dataset.split_by_classes(train_size=base_num_classes,
                                                                 random_state=seed)

Split by classes with train size = 4112 (seed = 11)
Train classes: 4112
Test classes: 2380
Train data: 82240 samples
Test data:  47600 samples


/mnt/disk/venv/dpy3/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Create few-shot dataset:

In [16]:
val_dataset, fewshot_dataset = val_fewshot_dataset.split_by_classes(train_size=val_num_classes,
                                                                    random_state=seed)

Split by classes with train size = 688 (seed = 11)
Train classes: 688
Test classes: 1692
Train data: 13760 samples
Test data:  33840 samples


## Backbone

In [17]:
model_name = 'baseline'

Create model:

In [18]:
backbone = ConvNet(input_size=(img_width, img_height, img_depth))

Instructions for updating:
Colocations handled automatically by placer.


Train model:

In [19]:
backbone.set_trainable(True)

In [20]:
backbone_optimizer = tf.keras.optimizers.Adam(lr=1e-3)

In [ ]:
simple_one_layer_cross_entropy_train(
    backbone,
    backbone_dataset.get_batch_generator(batch_size=backbone_training_batch_size,
                                         shuffle=True,
                                         generator_args=backbone_generator_args),
    epochs=backbone_training_epochs,
    optimizer=backbone_optimizer,
    model_name=model_name,
    checkpoint_dir=checkpoint_dir,
    period=5,
    tensorboard=True,
    log_dir=log_dir,
    use_multiprocessing=True,
    workers=4,
    verbose=2
)

## Few-shot

In [22]:
episode_generator = fewshot_dataset.get_fewshot_generator(n_way=n_way, 
                                                          k_shot=1, 
                                                          query_size=query_size,
                                                          support_generator_args=support_generator_args,
                                                          query_generator_args=query_generator_args)

Train few-shot model:

In [23]:
fewshot_optimizer = tf.keras.optimizers.Adam(lr=1e-3) 

In [24]:
fewshot_model = BaselineFewShotModel(backbone, n_way, with_cosine=True)
accuracies = baseline_fewshot_test(model=fewshot_model,
                                   generator=episode_generator, 
                                   optimizer=fewshot_optimizer,
                                   batch_size=fewshot_batch_size,
                                   support_epochs=support_epochs,
                                   n_episodes=n_episodes,
                                   model_name='baseline-fewshot',
                                   tensorboard=True,
                                   log_dir=log_dir,
                                   period=10)

Average acc: 90.62%: 100%|██████████| 30/30 [03:40<00:00,  8.75s/it]


# Report

Calculate 95% confidence interval:

In [25]:
mean_accuracy = np.mean(accuracies)
confidence_interval = stats.t.interval(0.95, len(accuracies) - 1, loc=mean_accuracy, scale=stats.sem(accuracies))

print("{}-way {}-shot metric: {}% +- {}%".format(
    n_way, k_shot,
    round(mean_accuracy * 100, 2),
    round((confidence_interval[1] - confidence_interval[0]) / 2 * 100, 2))
)

5-way 5-shot metric: 90.62% +- 3.32%
